In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import csv
import itertools
import re

# loading a viable scrabble words dictionary and board basics
# dictionary from https://raw.githubusercontent.com/jmlewis/valett/master/scrabble/sowpods.txt
data_path = "../sowpods.txt"
with open(data_path, newline="") as f:
    reader = csv.reader(f)
    allwords = list(reader)
WORDLIST = list(itertools.chain.from_iterable(allwords))
WORDLIST = [x.lower() for x in WORDLIST]
WORDSET = set(WORDLIST)
# making a dictionary with the keys as word length
print(max(len(ele) for ele in WORDLIST))

BOARD_LENGTH = 15
VOWELS = ["a", "e", "i", "o", "u"]
LETTER_MULTIPLIERS = [
    [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 0
    [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],  # 1
    [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 2
    [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 3
    [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 4
    [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
    [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 6
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 8
    [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
    [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 10
    [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],
    [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 12
    [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
    [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 14
]
LETTER_MULTIPLIERS_ARRAY = np.array(LETTER_MULTIPLIERS)

WORD_MULTIPLIERS = [
    [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 0
    [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],  # 1
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 2
    [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],  # 3
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 4
    [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 6
    [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 8
    [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 10
    [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 12
    [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],
    [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 14
]
WORD_MULTIPLIERS_ARRAY = np.array(WORD_MULTIPLIERS)

TILE_SCORES = {
    "a": 1,
    "b": 4,
    "c": 4,
    "d": 2,
    "e": 1,
    "f": 4,
    "g": 3,
    "h": 3,
    "i": 1,
    "j": 10,
    "k": 5,
    "l": 2,
    "m": 4,
    "n": 2,
    "o": 1,
    "p": 4,
    "q": 10,
    "r": 1,
    "s": 1,
    "t": 1,
    "u": 2,
    "v": 5,
    "w": 4,
    "x": 8,
    "y": 3,
    "z": 10,
}


15


<IPython.core.display.Javascript object>

In [3]:
# setting up the example conditions
sample_board = [
    ["", "", "", "", "", "", "", "", "", "", "", "e", "", "", ""],  # 0
    ["", "", "", "", "", "", "", "", "", "z", "o", "n", "a", "", "w"],  # 1
    ["", "", "", "", "", "", "", "", "", "", "", "a", "", "", "i"],  # 2
    ["", "", "", "", "", "", "", "", "", "r", "i", "m", "a", "", "l"],  # 3
    ["", "", "", "", "", "", "", "", "", "", "", "o", "b", "e", "y"],  # 4
    ["", "", "", "", "", "", "", "", "", "j", "u", "r", "a", "l", ""],
    ["", "", "", "", "", "", "", "b", "e", "e", "p", "", "", "", ""],  # 6
    ["", "", "", "", "", "", "y", "e", "w", "s", "", "t", "", "", ""],
    ["", "", "", "", "", "", "", "l", "e", "t", "c", "h", "", "", ""],  # 8
    ["", "", "", "", "h", "u", "r", "l", "", "", "", "i", "", "r", ""],
    ["", "", "", "", "", "", "e", "", "", "", "", "g", "", "e", ""],  # 10
    ["", "", "", "", "", "", "d", "", "", "", "", "h", "u", "e", "d"],
    ["", "", "", "", "", "", "", "", "", "", "", "", "", "f", ""],  # 12
    ["", "", "", "", "", "", "", "", "", "v", "e", "e", "p", "s", ""],
    ["", "", "", "", "", "", "", "", "n", "u", "n", "", "", "", ""],  # 14
]
sample_board_letters = ["o", "d", "a", "s"]  # "d", "d", "a", "r"]

# numpy of the board
board_array = np.array(sample_board)

# making the empty strings spaces
neat_sample_array = np.copy(board_array)
neat_sample_array[neat_sample_array == ""] = " "
neat_sample_array_orig = np.copy(neat_sample_array)  # dont need this
T_neat_array = np.copy(neat_sample_array)
T_neat_array = T_neat_array.transpose()
print("transpose array:")
print(T_neat_array)
print("normal array:")
print(neat_sample_array)
# could think about thinking about potential words as a third axis

transpose array:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'u' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'y' ' ' 'r' 'e' 'd' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'b' 'e' 'l' 'l' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'e' 'w' 'e' ' ' ' ' ' ' ' ' ' ' 'n']
 [' ' 'z' ' ' 'r' ' ' 'j' 'e' 's' 't' ' ' ' ' ' ' ' ' 'v' 'u']
 [' ' 'o' ' ' 'i' ' ' 'u' 'p' ' ' 'c' ' ' ' ' ' ' ' ' 'e' 'n']
 ['e' 'n' 'a' 'm' 'o' 'r' ' ' 't' 'h' 'i' 'g' 'h' ' ' 'e' ' ']
 [' ' 'a' ' ' 'a' 'b' 'a' ' ' ' ' ' ' ' ' ' ' 'u' ' ' 'p' ' ']
 [' ' ' ' ' ' ' ' 'e' 'l' ' ' ' ' ' ' 'r' 'e' 'e' 'f' 's' ' ']
 [' ' 'w' 'i' 'l' 'y' ' ' ' ' ' ' ' ' ' ' ' ' 'd' ' ' ' ' ' ']]
normal array:
[[' ' ' ' ' ' ' ' ' ' '

<IPython.core.display.Javascript object>

## Finding valid words ##

In [4]:
# make array that lists number of empty tiles, then the word on the board (as a pattern)
def surroundings_list2(board):
    words_in_board = []
    lines_in_board = []

    for row in board:  # use combo of itertools and list comprehension
        rowstring = ["".join(x) for _, x in itertools.groupby(row, key=str.isalpha)]
        rowlist = [len(w) if " " in w else w for w in rowstring]

        # inserting 0s on the ends if there are tiles on the edge
        if type(rowlist[0]) == str:
            rowlist = [0] + rowlist
        if type(rowlist[-1]) == str:
            rowlist = rowlist + [0]
        words_in_board.append(rowlist)
    return words_in_board


<IPython.core.display.Javascript object>

In [5]:
# helper functions to see if the tiles are filled right and left of coordinates
# y and x are the x coordinates of the tile to be checked
def check_right(array, y, x):
    if x < BOARD_LENGTH - 1:
        return array[y, x + 1] != " ", array[y, x + 1]
    else:
        return "out"


def check_left(array, y, x):
    if x > 0:
        return array[y, x - 1] != " ", array[y, x - 1]
    else:
        return "out"


def check_above(array, y, x):
    if y > 0:
        return array[y - 1, x] != " ", array[y - 1, x]
    else:
        return "out"


def check_below(array, y, x):
    if y < BOARD_LENGTH - 1:
        return array[y + 1, x] != " ", array[y + 1, x]
    else:
        return "out"


def crossing_word_tile(board, letter_crossed, y, x):
    Y = y
    Y_i = y
    X = x
    above_word = ""
    below_word = ""
    # seeing if there are crosses above and below and if the tile is already on the board
    while check_above(board, y, x)[0] == True:
        above_word = above_word + check_above(board, y, x)[1]
        y = y - 1
        Y_i = y
    above_word = above_word[::-1]
    y = Y
    while check_below(board, y, x)[0] == True:
        below_word = below_word + check_below(board, y, x)[1]
        y = y + 1

    crossing_word = above_word + letter_crossed + below_word
    crossing_word_index = (crossing_word, Y_i, x)
    return crossing_word_index


# word_crossed below has the form [word, y index, x index]
def new_crossing_words_word(array, word_crossed):
    print(word_crossed)
    y = word_crossed[1]
    x = word_crossed[2]
    crossing_words = set()
    count = 0
    for letter in word_crossed[0]:
        if (
            check_above(array, y, x)[0] == True or check_below(array, y, x)[0] == True
        ) and array[y, x] != letter:

            crossing_word = crossing_word_tile(array, letter, y, x)
            crossing_words.add(crossing_word)

        if array[y, x] != letter:
            count = count + 1
            edge_letter = letter
            edge_x = x
            edge_y = y

        x = x + 1

    if count == 1 and len(crossing_words) == 0:
        crossing_words.add((edge_letter, edge_y, edge_x))
    if len(crossing_words) == 0:
        crossing_words.add(("", -1, -1))
    return crossing_words

<IPython.core.display.Javascript object>

In [6]:
# first, find the valid words, then filter out which have left/middle/right word too far from the outside
# as determined by 1-the number infront/behind it. 

def valid_main_words(spaces_array, hand_letters):
    valid_words = []
    all_potentials_dict={}
    row_index = -1
    for row in spaces_array:
        row_index = row_index + 1
        valid_row_words = []
        
        #make sure there are words in the row
        if len(row) > 1:
            
            #row words is a list of just the words in the row
            board_row_words = row[1::2]

            #looping through each WORD in the ROW
            #the count is the index on a row of the spaces_array
            count = 0
            for board_row_word in board_row_words:
                #these are the indexes for row[index], indicating just to the left and right of the ROW WORD,
                #as well as the ROW WORD itself
                left_index = count
                right_index = count + 2
                word_index = count + 1
                
                #finding the index of the BOARD WORD from the left on the original board
                left_tile_index = sum(row[0:right_index:2]) + sum(
                    [len(w) for w in board_row_words[0 : count // 2 + 1]])-len(board_row_word)
                
                potentials_dict=valid_words_around_board_word(row, board_row_word, hand_letters, count, row_index, left_tile_index)
                count = count + 2
                if len(potentials_dict)>0:
                    #valid_row_words=valid_row_words+potential_words
                    for k in potentials_dict:
                        if k in all_potentials_dict:
                            all_potentials_dict[k].append(potentials_dict[k])
                        else:
                            all_potentials_dict[k] = [potentials_dict[k]]
    valid_words=potentials_dict_to_valids(all_potentials_dict)

    return valid_words


<IPython.core.display.Javascript object>

In [7]:

# first, find the valid words, then filter out which have left/middle/right word too far from the outside
# as determined by 1-the number infront/behind it. 

def valid_words_across2(board, hand_letters):
    spaces_array = surroundings_list2(board)
    valid_words= valid_main_words(spaces_array, hand_letters)
    
    #now I want to check if the crosses are valid
    valid_words_crossed=[]
    #generating all the crossing words
    # word is in form ['word', y, x], x and y started at 0, y=row, x=column/index in row
    for valid_word in valid_words:
        #debugging
        crossing_words_index=new_crossing_words_word(board, valid_word)
        #crossing_words= [i[0] for i in crossing_words_index if i[0] != ()]
        crossing_words=list(zip(*crossing_words_index))[0]
        crossing_words=set(crossing_words)
        if crossing_words.issubset(WORDSET) or len(list(crossing_words)[0])<2:
            valid_words_crossed= valid_words_crossed + [[list(valid_word)]+[list(crossing_words_index)]]
    return valid_words_crossed
    #return both the word and its crossing words, will make it easier to score



<IPython.core.display.Javascript object>

In [8]:
#TODO: special cases where there's only 1 letter used for across, can go the other way.

def valid_words_whole2(board, transpose_board, hand_letters):
    valid_row_words = valid_words_across2(board, hand_letters)
    # remember the row and column indexes will be returned transposed for the columns
    valid_column_words = valid_words_across2(transpose_board, hand_letters)
    edge_column_words=valid_edge_words(transpose_board, valid_row_words, hand_letters,)
    edge_row_words=valid_edge_words(board, valid_column_words, hand_letters,)
    
    valid_row_words=valid_row_words+edge_row_words
    valid_column_words=valid_column_words+edge_column_words
    return valid_row_words, valid_column_words


<IPython.core.display.Javascript object>

Concerns: 

check out regex search with []boardstring[][][] type thing
or itertools, specifically itertool.combinations
or something like a combo of the two:

.....boardstring...

then filter that based on possible combos

## Scoring ##

In [9]:
def calculate_word_score(valid_word, letter_multipliers_array, word_multipliers_array):
    # word should be in the form ['word', y, x]
    word = valid_word[0]
    y = valid_word[1]
    x = valid_word[2]
    letter_multipliers = np.copy(letter_multipliers_array[y, x : x + len(word)])
    word_multipliers = np.copy(word_multipliers_array[y, x : x + len(word)])
    scores = [TILE_SCORES[l] for l in word]
    word_score = np.dot(letter_multipliers, scores)
    word_score = word_score * np.prod(word_multipliers)
    return word_score


# output of valid_words2 should be in the form [['word', y, x], [('c_word1', y, x), ('c_word2', y, x), ...]]
# words with no crosses have the form [[word, y, x], [('', -1, -1)]]
# (this goes int the valid_word_with_crosses spot)
# when calculating the crosses, the board used should be the transpose


def total_score(
    board, valid_word_with_crosses, letter_multipliers_array, word_multipliers_array
):
    # make the  multipliers==1 if there are pieces on them on the board;
    # TODO: how permanent is the below?
    bool_mask = (board != " ").astype(int)
    letter_multipliers_array = np.where(bool_mask == 1, 1, letter_multipliers_array)
    word_multipliers_array = np.where(bool_mask == 1, 1, word_multipliers_array)

    main_word = valid_word_with_crosses[0]
    scored_crossing_words = valid_word_with_crosses[1]

    main_score = calculate_word_score(
        main_word, letter_multipliers_array, word_multipliers_array
    )
    # TODO: ask if I should have transpose inside or outside the copy function here
    # transpose the multiplier arrays to use to score the crossing words
    T_word_multipliers = np.copy(word_multipliers_array.transpose())
    T_letter_multipliers = np.copy(letter_multipliers_array.transpose())

    # calculating the crossing words' total score
    # need to switch the x and y here so the words don't go off the board
    crossers_score = 0
    for crosser in scored_crossing_words:
        if len(crosser[0]) > 1:
            crosser = list(crosser)
            crosser[1], crosser[2] = crosser[2], crosser[1]
            crosser_score = calculate_word_score(
                crosser, T_letter_multipliers, T_word_multipliers
            )
        else:
            crosser_score = 0
        crossers_score = crossers_score + crosser_score
    total_score = main_score + crossers_score
    return total_score


def valid_score_choices(
    board, hand_letters, letter_multiplier_array, word_multiplier_array
):
    # make a transpose board
    transpose_board = np.copy(board)
    transpose_board = transpose_board.transpose()

    # calculate the words/placement allowed by the rules of the game
    valid_across_words_crosses, valid_column_words_crosses = valid_words_whole2(
        board, transpose_board, hand_letters
    )

    # create a dictionary of each of these words along with their score
    across_score_dict = {}
    column_score_dict = {}
    for word_crosses in valid_across_words_crosses:
        word = word_crosses[0]
        score = total_score(
            board, word_crosses, letter_multiplier_array, word_multiplier_array
        )
        across_score_dict[tuple(word)] = score

    for word_crosses in valid_column_words_crosses:
        column_word = word_crosses[0]
        column_score = total_score(
            transpose_board,
            word_crosses,
            letter_multiplier_array,
            word_multiplier_array,
        )
        column_score_dict[tuple(column_word)] = column_score
    return across_score_dict, column_score_dict


# the across scores are going to be returned with the format ('word',y,x)
# while the column scores will be returned as ('word',x,y)

<IPython.core.display.Javascript object>

In [10]:
def best_plays(board, hand_letters, letter_multiplier_array, word_multiplier_array):
    across_score_dict, column_score_dict=valid_score_choices(
    board, hand_letters, letter_multiplier_array, word_multiplier_array
)
    top_across_score=list(across_score_dict.values())
    top_across_word=list(across_score_dict.keys())
    
    top_column_score=list(column_score_dict.values())
    top_column_word=list(column_score_dict.keys())

    best_across_word_index= top_across_word[top_across_score.index(max(top_across_score))] 
    best_column_word_index=top_column_word[top_column_score.index(max(top_column_score))]
    best_across_dict={}
    best_column_dict={}
    best_across_dict[best_across_word_index]=max(top_across_score)
    best_column_dict[best_column_word_index]=max(top_column_score)
    return best_across_dict, best_column_dict


<IPython.core.display.Javascript object>

### Edge Cases: ###

In [11]:
# edge letters will be in format [['char', y, x],...] but y and x will be from the transpose board, so switch them
# do something like:
# for letter in edge letters, make a new board with that letter on it
# board will be transposed here from when we formed the 2 letter orignal word
def valid_edge_words(board, valid_word_across_output, hand_letters):
    valid_words_crossed = []
    for valid_word in valid_word_across_output:
        edge_letter = valid_word[1][0][0]
        original_word = valid_word[0][0]
        # x and y are switched because I'm getting this from the other direction
        x = valid_word[1][0][1]
        y = valid_word[1][0][2]
        o_y = valid_word[0][1]
        o_x = valid_word[0][2]
        board_row = board[y]
        # filter for if the edge is only 1 letter. This means the word is eligible as an edge word
        # the original word will later be added as a crossing word (x and y transposed)
        if len(edge_letter) == 1:
            # debugging
            print("VW")
            print(valid_word)
            edge_board = np.copy(
                board
            )  # this board is transposed from earlier, passed in here
            edge_board[y, x] = edge_letter

            # add the edge_word back in at the end as a crossing word
            spaces_row = surroundings_list2(edge_board)[y]
            edge_hand_letters = hand_letters[:]
            edge_hand_letters.remove(edge_letter)
            spaces_row_length = spaces_row[:]
            spaces_row_length[1::2] = [len(w) for w in spaces_row_length[1::2]]
            s_r_acc = list(itertools.accumulate(spaces_row_length))
            count = 0
            for acc in s_r_acc:
                if acc < x + 1:
                    count = count + 1
            count = count - 1
            potentials_dict = valid_words_around_board_word(
                spaces_row, edge_letter, edge_hand_letters, count, y, x
            )
            valid_words = potentials_dict_to_valids(potentials_dict)
            # now I want to check if the crosses are valid
            # generating all the crossing words
            # word is in form ['word', y, x], x and y started at 0, y=row, x=column/index in row
            for valid_word in valid_words:
                crossing_words_index = new_crossing_words_word(board, valid_word)
                crossing_words_index.add(
                    (original_word, o_x, o_y)
                )  # should x and y be crossed here? think so.
                # crossing_words= [i[0] for i in crossing_words_index if i[0] != ()]
                crossing_words = list(zip(*crossing_words_index))[0]
                crossing_words = set(crossing_words)
                if crossing_words.issubset(WORDSET) or crossing_words == {""}:
                    valid_words_crossed = valid_words_crossed + [
                        [list(valid_word)] + [list(crossing_words_index)]
                    ]
    return valid_words_crossed


# spaces_row, board_row_word, hand_letters, count, row_index, left_tile_index

<IPython.core.display.Javascript object>

In [12]:
# could change it so the parameters are the brw, what's left of it, what's right of it, and the other stuff
def side_permutation(perm, spaces_numbers, side_words):
    chunks = [
        perm[sum(spaces_numbers[:i]) : sum(spaces_numbers[: i + 1])]
        for i in range(len(spaces_numbers))
    ]

    side_permutation = [
        x
        for x in itertools.chain.from_iterable(
            itertools.zip_longest(chunks, side_words)
        )
        if x
    ]
    side_permutation = "".join(side_permutation)
    return side_permutation


def valid_words_around_board_word(
    spaces_row, board_row_word, hand_letters, count, row_index, left_tile_index
):
    potentials_dict = {}
    # WORDDICT = {key: [] for key in WORDSET}
    spaces_row_length = len(spaces_row)
    spaces_word_index = count + 1
    words_to_left, words_to_right = [], []
    if spaces_word_index > 1:
        words_to_left = spaces_row[spaces_word_index - 2 :: -2]
    if spaces_word_index < spaces_row_length:
        words_to_right = spaces_row[spaces_word_index + 2 :: 2]
    spaces_to_left = spaces_row[spaces_word_index - 1 :: -2]
    spaces_to_right = spaces_row[spaces_word_index + 1 :: 2]
    # going to use this to pre-filter the potential words list
    #     potential_words = {
    #         word
    #         for word in WORDSET
    #         if board_row_word in word and len(word) > len(board_row_word)
    #     }

    # combining the board words and your hand letters into one list, then finding the combinations
    all_letters = [board_row_word] + hand_letters
    letter_perms = [
        "".join(l)
        for i in range(len(all_letters))
        for l in itertools.permutations(all_letters, i + 1)
    ]
    # then filtering it
    perms_containing_brw = [
        p for p in letter_perms if board_row_word in p and len(p) > len(board_row_word)
    ]
    potential_perms = []
    potential_perms_with_position = set()
    words_and_positions = {}
    for perm in perms_containing_brw:
        find_index = -1
        find_indexes = []
        while perm.find(board_row_word, find_index + 1) > -1:

            # find a index of the board row word in the perm
            find_index = perm.find(board_row_word, find_index + 1)
            find_indexes = find_indexes + [find_index]

            # first the left
            # count number of spaces you'll need
            left_perm = perm[0:find_index]
            left_perm_rev = left_perm[::-1]
            l_c = 0  # c is number of times the letters will touch words to the left
            try:
                while len(left_perm) >= np.cumsum(spaces_to_left)[l_c]:
                    # leftover_spaces_l = len(left_perm) - np.cumsum(spaces_to_left)[l_c]
                    l_c = l_c + 1
                    if len(left_perm) == np.cumsum(spaces_to_right)[l_c - 1]:
                        break
            except:
                continue
            words_touched_l = words_to_left[0:l_c]
            words_touched_l = [w[::-1] for w in words_touched_l]
            potential_perm_l = side_permutation(
                left_perm_rev, spaces_to_left, words_touched_l
            )
            potential_perm_l = potential_perm_l[::-1]

            # now the right
            right_perm = perm[find_index:]
            r_c = 0  # c is number of times the letters will touch words to the right
            try:
                while len(right_perm) >= np.cumsum(spaces_to_right)[r_c]:
                    # leftover_spaces_r = len(right_perm) - np.cumsum(spaces_to_right)[r_c]
                    r_c = r_c + 1
                    if len(right_perm) == np.cumsum(spaces_to_right)[r_c - 1]:
                        break
            except:
                continue
            words_touched_r = words_to_right[0:r_c]
            potential_perm_r = side_permutation(
                right_perm, spaces_to_right, words_touched_r
            )

            potential_perm = potential_perm_l + potential_perm_r
            pp_length = len(potential_perm)

            # TODO remember to check if the word goes off the side, or is this automatic?
            #   if left_tile_index-len(potential_perm_l)>=0 and left_tile_index+len(potential_perm_r)<=14:

            potential_perms = potential_perms + [potential_perm]
            potential_perm_with_position = (
                potential_perm,
                row_index,
                left_tile_index - len(potential_perm_l),
            )  # I think the above with the len() part is correct
            potential_perms_with_position.add(potential_perm_with_position)
            if potential_perm in potentials_dict:
                potentials_dict[potential_perm].append(potential_perm_with_position)
            else:
                potentials_dict[potential_perm] = [potential_perm_with_position]
    #             try:
    #                 WORDDICT[potential_perm].append(potential_perm_with_position)
    #             except:
    #                 break
    #             potential_perms_with_position = potential_perms_with_position + [
    #                 potential_perm_with_position
    #             ]

    # potential_perms = set(potential_perms)
    #     values = []
    #     for value in WORDDICT.values():
    #         value = tuple(value)
    #         values.append(value)
    #     poslist = set(values)

    #     valid_brw_words = [item for sublist in poslist for item in sublist]

    #     valid_brw_words = [
    #         p_p_w_p
    #         for p_p_w_p in potential_perms_with_position
    #         if p_p_w_p[0] in potential_words
    #     ]
    return potentials_dict

<IPython.core.display.Javascript object>

In [13]:
def potentials_dict_to_valids(potentials_dictionary):
    potentials = set(potentials_dictionary.keys())
    potentials_with_position = []
    valid_plays = [
        potentials_dictionary[word] for word in WORDSET if word in potentials
    ]
    flat_list = [item for sublist in valid_plays for item in sublist]
    flatter_list = [item for sublist in flat_list for item in sublist]
    # debugging
    # print(flatter_list)
    print("done")
    return flatter_list

<IPython.core.display.Javascript object>

In [14]:
print("transpose array:")
print(T_neat_array)
print("normal array:")
print(neat_sample_array)

transpose array:
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'u' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'y' ' ' 'r' 'e' 'd' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'b' 'e' 'l' 'l' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'e' 'w' 'e' ' ' ' ' ' ' ' ' ' ' 'n']
 [' ' 'z' ' ' 'r' ' ' 'j' 'e' 's' 't' ' ' ' ' ' ' ' ' 'v' 'u']
 [' ' 'o' ' ' 'i' ' ' 'u' 'p' ' ' 'c' ' ' ' ' ' ' ' ' 'e' 'n']
 ['e' 'n' 'a' 'm' 'o' 'r' ' ' 't' 'h' 'i' 'g' 'h' ' ' 'e' ' ']
 [' ' 'a' ' ' 'a' 'b' 'a' ' ' ' ' ' ' ' ' ' ' 'u' ' ' 'p' ' ']
 [' ' ' ' ' ' ' ' 'e' 'l' ' ' ' ' ' ' 'r' 'e' 'e' 'f' 's' ' ']
 [' ' 'w' 'i' 'l' 'y' ' ' ' ' ' ' ' ' ' ' ' ' 'd' ' ' ' ' ' ']]
normal array:
[[' ' ' ' ' ' ' ' ' ' '

<IPython.core.display.Javascript object>


\

In [15]:
# can take 10 minutes to do. How do I shorten this?
valid_score_choices(
    neat_sample_array,
    sample_board_letters,
    LETTER_MULTIPLIERS_ARRAY,
    WORD_MULTIPLIERS_ARRAY,
)
# best_plays(
#     neat_sample_array,
#     sample_board_letters,
#     LETTER_MULTIPLIERS_ARRAY,
#     WORD_MULTIPLIERS_ARRAY,
# )

done
('sado', 11, 4)
('sado', 11, 4)
('ose', 0, 9)
('ose', 10, 4)
('dof', 12, 11)
('dods', 11, 4)
('dods', 11, 4)
('de', 0, 10)
('de', 10, 5)
('ados', 11, 5)
('ados', 11, 5)
('ed', 0, 11)
('ed', 10, 6)
('tas', 7, 11)
('does', 0, 9)
('does', 10, 4)
('sed', 0, 10)
('sed', 10, 5)
('do', 11, 6)
('do', 11, 6)
('ado', 11, 5)
('ado', 11, 5)
('sad', 11, 4)
('sad', 11, 4)
('aged', 10, 10)
('ages', 10, 10)
('ods', 11, 5)
('ods', 11, 5)
('ode', 0, 9)
('ode', 10, 4)
('das', 11, 6)
('das', 11, 6)
('age', 10, 10)
('ad', 11, 5)
('ad', 11, 5)
('airs', 9, 10)
('eds', 0, 11)
('eds', 10, 6)
('of', 12, 12)
('sade', 0, 8)
('sade', 10, 3)
('adds', 11, 5)
('adds', 11, 4)
('adds', 11, 5)
('adds', 11, 4)
('sage', 10, 9)
('daes', 0, 9)
('daes', 10, 4)
('odas', 11, 5)
('odas', 11, 5)
('dados', 11, 4)
('dados', 11, 4)
('gae', 10, 11)
('odd', 11, 5)
('odd', 11, 4)
('odd', 11, 5)
('odd', 11, 4)
('soda', 2, 8)
('soda', 2, 8)
('soda', 11, 4)
('soda', 11, 4)
('sadi', 2, 10)
('sadi', 2, 10)
('dso', 11, 6)
('dso', 11, 6

IndexError: string index out of range

<IPython.core.display.Javascript object>

In [ ]:
WORDDICT = {key: [] for key in WORDSET}
values = []
for value in WORDDICT.values():
    value = tuple(value)
    values.append(value)
poslist = set(values)
[item for sublist in poslist for item in sublist]


I should make a "check words function" in which I pass in a dictionary of potential words. Then I go through the dictionary, checking if the words are in the keys of the dictionary (or the other way around)


In [ ]:
any(isinstance(el, list) for el in input_list)
